In [62]:
# KNN WITHOUT ANY PREPROCESSING

In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
new_df=pd.read_csv('../raw_data/cleaned_data_040321.csv',encoding='latin-1')

In [74]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [65]:
y=new_df[['neu','ext','ope','agr','con']]
X=new_df.loc[:,'BFF_15_1':'BFF_15_15']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=0)

In [66]:
knn=KNeighborsRegressor(n_neighbors=15)

In [67]:
predictions=cross_val_predict(knn,X,y,cv=5)

In [21]:
new_df[['neu_pred','ext_pred','ope_pred','agr_pred','con_pred']]=predictions

In [68]:
knn.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                    weights='uniform')

In [69]:
y_pred=knn.predict(X_test)

In [70]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)

0.044364059575269815

In [71]:
knn.score(X_test,y_test)

/home/lpshruthi/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/base.py:426: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  warnings.warn("The default value of multioutput (not exposed in "


0.9523105260056557

# now need to do pipelining

In [215]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

new_df=pd.read_csv('../raw_data/cleaned_data_040321.csv',encoding='latin-1')
X=new_df[['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11',
           'BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat',
          'Dem_maritalstatus','Dem_riskgroup','Dem_isolation'
         ]]


In [220]:
def edu_func(X):
    X['Dem_edu']=X['Dem_edu'].replace({'Uninformative response':0,'None':1,'Up to 6 years of school':2, 'Up to 9 years of school':3, 'Up to 12 years of school':4, 'Some College, short continuing education or equivalent':5, 'College degree, bachelor, master': 6, 'PhD/Doctorate':7 })
    return  X[['Dem_edu']]

    
def edu_mom_func(X):    
    X['Dem_edu_mom'] = X['Dem_edu_mom'].replace({'Uninformative response':0,'None':1,'Up to 6 years of school':2, 'Up to 9 years of school':3, 'Up to 12 years of school':4, 'Some College or equivalent':5, 'College degree': 6, 'PhD/Doctorate':7 })
    return  X[['Dem_edu_mom']]


def edu_risk_group(X): 
    X['Dem_riskgroup'] = X['Dem_riskgroup'].replace({'No':1,'Not sure':2, 'Yes':3})    
    return  X[['Dem_riskgroup']]

def dem_expat_func(X):
    X['Dem_Expat'] = X['Dem_Expat'].replace({'no':0,'yes':1})
    return X[['Dem_Expat']]
    



imputing = ColumnTransformer([
   
    ('Dem_edu_col', FunctionTransformer(edu_func), ['Dem_edu']),
    ('Dem_edu_mom', FunctionTransformer(edu_mom_func), ['Dem_edu_mom']),
    ('Dem_edu_risk_group', FunctionTransformer(edu_risk_group), ['Dem_riskgroup']),
    ('dem_expat_func', FunctionTransformer(dem_expat_func), ['Dem_Expat']),
    ('ohe_transformer', OneHotEncoder(), ['Dem_gender', 'Dem_maritalstatus','Dem_employment','Dem_isolation'])
],remainder='passthrough')

final_pipe = Pipeline([

 
    ('imputer', imputing),
    ('scaler', MinMaxScaler())
])


final_pipe


Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Dem_edu_col',
                                                  FunctionTransformer(func=<function edu_func at 0x7fc6e5e2a700>),
                                                  ['Dem_edu']),
                                                 ('Dem_edu_mom',
                                                  FunctionTransformer(func=<function edu_mom_func at 0x7fc6e5e2a5e0>),
                                                  ['Dem_edu_mom']),
                                                 ('Dem_edu_risk_group',
                                                  FunctionTransformer(func=<function edu_risk_group at 0x7fc6e5e2a550>),
                                                  ['Dem_riskgroup']),
                                                 ('dem_expat_func',
                                                  FunctionTransformer(func=<function dem_expat_func at 0x7fc6e5e2a820>),
                                                  ['Dem_Expat']),
                                                 ('ohe_transformer',
                                                  OneHotEncoder(),
                                                  ['Dem_gender',
                                                   'Dem_maritalstatus',
                                                   'Dem_employment',
                                                   'Dem_isolation'])])),
                ('scaler', MinMaxScaler())])

In [217]:
pd.set_option('display.max_columns', None)

In [218]:
X=new_df[['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11',
           'BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat',
          'Dem_maritalstatus','Dem_riskgroup','Dem_isolation'
         ]]


In [422]:
#scaled_X=pd.DataFrame(final_pipe.fit_transform(X)).head(10)
#scaled_X


# KNN FILE organizing into files:

In [468]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error




def get_user_data():
    values=[]

    columns=['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11','BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15']

    for i in columns:
        value=random.randint(1,6)
        values.append(value)
    
    user_data=pd.DataFrame([values],columns=columns)
    return user_data

def get_data():
    return pd.read_csv('../raw_data/cleaned_data_040321.csv')

def Xandy(df):
    X=df.loc[:,'BFF_15_1':'BFF_15_15']
    y=df[['neu','ext','ope','agr','con']]
    return X,y

def model(X,y):
    knn=KNeighborsRegressor(n_neighbors=15)
    return knn.fit(X,y)
    

def predict(model):
    
    y_pred=model.predict(user_data)
    return y_pred


In [ ]:
#### MY MAIN FILE

In [469]:
train_data=get_data()
X_train,y_train=Xandy(train_data)
X_test=get_user_data()
knn=model(X_train,y_train)


In [473]:
import joblib
joblib.dump(knn,'knn.joblib')

['knn.joblib']

In [470]:
y_pred=knn.predict(X_test)

In [471]:
y_pred

array([[2.44444438, 3.8       , 4.        , 4.37777776, 4.02222222]])

#to make df
import random
import pandas as pd
import numpy as np
values=[]

columns=['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11','BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15']

for i in columns:
    value=random.randint(1,6)
    values.append(value)
    
data=pd.DataFrame([values],columns=columns)
data

In [426]:
from sklearn import set_config; set_config(display='diagram')
final_pipe

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Dem_edu_col',
                                                  FunctionTransformer(func=<function edu_func at 0x7fc6e5e2a700>),
                                                  ['Dem_edu']),
                                                 ('Dem_edu_mom',
                                                  FunctionTransformer(func=<function edu_mom_func at 0x7fc6e5e2a5e0>),
                                                  ['Dem_edu_mom']),
                                                 ('Dem_edu_risk_group',
                                                  FunctionTransformer(func=<function edu_risk_group at 0x7fc6e5e2a550>),
                                                  ['Dem_riskgroup']),
                                                 ('dem_expat_func',
                                                  FunctionTransformer(func=<function dem_expat_func at 0x7fc6e5e2a820>),
                                                  ['Dem_Expat']),
                                                 ('ohe_transformer',
                                                  OneHotEncoder(),
                                                  ['Dem_gender',
                                                   'Dem_maritalstatus',
                                                   'Dem_employment',
                                                   'Dem_isolation'])])),
                ('scaler', MinMaxScaler())])

# PREDICT STRESS , PACKAGING

In [132]:
!pip install mord

  Created wheel for mord: filename=mord-0.6-py3-none-any.whl size=6006 sha256=e9dff3cf599182f9cd6eb64fa004a3b7f5ba4cbe30702fc5235aa32acf9db382
  Stored in directory: /home/lpshruthi/.cache/pip/wheels/1d/94/48/b3d11f5f91d26bec0c95c6eb9c27c6e427d0b77fb2406d0e0a
Successfully built mord


In [460]:

def get_data():
    return pd.read_csv('../raw_data/cleaned_data_040321.csv')

def Xandy_stress(df):
    X=df[['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11',
           'BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat',
          'Dem_maritalstatus','Dem_riskgroup','Dem_isolation'
         ]].reset_index(drop=True).dropna()
    
    y=df['PSS10_avg'].round().astype(int)
    return X,y    

In [467]:
def edu_func(X):
    X['Dem_edu']=X['Dem_edu'].replace({'Uninformative response':0,'None':1,'Up to 6 years of school':2, 'Up to 9 years of school':3, 'Up to 12 years of school':4, 'Some College, short continuing education or equivalent':5, 'College degree, bachelor, master': 6, 'PhD/Doctorate':7 })
    return  X[['Dem_edu']]

    
def edu_mom_func(X):    
    X['Dem_edu_mom'] = X['Dem_edu_mom'].replace({'Uninformative response':0,'None':1,'Up to 6 years of school':2, 'Up to 9 years of school':3, 'Up to 12 years of school':4, 'Some College or equivalent':5, 'College degree': 6, 'PhD/Doctorate':7 })
    return  X[['Dem_edu_mom']]


def edu_risk_group(X): 
    X['Dem_riskgroup'] = X['Dem_riskgroup'].replace({'No':1,'Not sure':2, 'Yes':3})    
    return  X[['Dem_riskgroup']]

def dem_expat_func(X):
    X['Dem_Expat'] = X['Dem_Expat'].replace({'no':0,'yes':1})
    return X[['Dem_Expat']]

In [462]:
def process_pipeline(X,model):
    
    

    
    imputing = ColumnTransformer([
        
   
    ('Dem_edu_col', FunctionTransformer(edu_func), ['Dem_edu']),
    ('Dem_edu_mom', FunctionTransformer(edu_mom_func), ['Dem_edu_mom']),
    ('Dem_edu_risk_group', FunctionTransformer(edu_risk_group), ['Dem_riskgroup']),
    ('dem_expat_func', FunctionTransformer(dem_expat_func), ['Dem_Expat']),
    ('ohe_transformer', OneHotEncoder(), ['Dem_gender', 'Dem_maritalstatus','Dem_employment','Dem_isolation'])
    ],remainder='passthrough')
    
    base_pipe = Pipeline([

 
    ('imputer', imputing),
    ('scaler', MinMaxScaler())
    ])
    

    
    stress_pipe = Pipeline([
    ('preprocessing', base_pipe),
    ('LogisticAT_regression',  model  )
    ])
    
    return stress_pipe


In [452]:
#def get_df(pipeline,X):
 #   import pandas as pd
    
  #  return pd.DataFrame(pipeline.fit_transform(X))

In [463]:
# MAIN FILE 
from mord import LogisticAT
import joblib
import pandas as pd
data=get_data()
X,y=Xandy_stress(data)

model=process_pipeline(X,LogisticAT(alpha=0))
model
model.fit(X,y)
#joblib.dump(model, 'model_stress.joblib')
#model.predict(X.loc[[0]]) # INPUT NEEDS TO BE A DATAFRAME
#scaled_X=get_df(stress_model,X)

#pd.DataFrame(model.fit_transform(X))

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('Dem_edu_col',
                                                                   FunctionTransformer(func=<function edu_func at 0x7fc765b91790>),
                                                                   ['Dem_edu']),
                                                                  ('Dem_edu_mom',
                                                                   FunctionTransformer(func=<function edu_mom_func at 0x7fc75af35a60>),
                                                                   ['Dem_edu_mom']),
                                                                  ('Dem_edu_risk_group',
                                                                   FunctionTransformer(func=<function edu_risk_group at 0x7fc75af35c10>),
                                                                   ['Dem_riskgroup']),
                                                                  ('dem_expat_func',
                                                                   FunctionTransformer(func=<function dem_expat_func at 0x7fc75af35ca0>),
                                                                   ['Dem_Expat']),
                                                                  ('ohe_transformer',
                                                                   OneHotEncoder(),
                                                                   ['Dem_gender',
                                                                    'Dem_maritalstatus',
                                                                    'Dem_employment',
                                                                    'Dem_isolation'])])),
                                 ('scaler', MinMaxScaler())])),
                ('LogisticAT_regression', LogisticAT(alpha=0))])

In [464]:

joblib.dump(model, 'model_stress.joblib')
#model.predict(X.loc[[0]]) # INPUT NEEDS TO BE A DATAFRAME

['model_stress.joblib']

In [466]:
stress_pred=joblib.load('model_stress.joblib')
stress_pred.predict(X.loc[[0]])

array([2])

In [416]:
X.loc[[0]]

,BFF_15_1,BFF_15_2,BFF_15_3,BFF_15_4,BFF_15_5,BFF_15_6,BFF_15_7,BFF_15_8,BFF_15_9,BFF_15_10,BFF_15_11,BFF_15_12,BFF_15_13,BFF_15_14,BFF_15_15,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,Dem_Expat,Dem_maritalstatus,Dem_riskgroup,Dem_isolation
0,2.0,2.0,5.0,5.0,5.0,2.0,5.0,6.0,5.0,2.0,4.0,6.0,5.0,2.0,5.0,20,Male,"College degree, bachelor, master",None,Student,yes,Single,Yes,Life carries on with minor changes


In [417]:
#X.iloc[-1]
#model.predict(pd.DataFrame(X.iloc[-1]).T)
model.predict(X.loc[[0]])

array([2])

# lONELINESS


In [ ]:
new_df['SLON3_avg']

In [437]:
def Xandy_lone(df):
    X=df[['BFF_15_1','BFF_15_2','BFF_15_3','BFF_15_4','BFF_15_5','BFF_15_6','BFF_15_7','BFF_15_8','BFF_15_9','BFF_15_10','BFF_15_11',
           'BFF_15_12','BFF_15_13','BFF_15_14','BFF_15_15','Dem_age','Dem_gender','Dem_edu','Dem_edu_mom','Dem_employment','Dem_Expat',
          'Dem_maritalstatus','Dem_riskgroup','Dem_isolation'
         ]].reset_index(drop=True).dropna()
    
    y=df['SLON3_avg'].round().astype(int)
    return X,y    

In [439]:
from mord import LogisticAT
import pandas as pd
data=get_data()
X,y=Xandy_lone(data)

model=process_pipeline(X,LogisticAT(alpha=0))

model.fit(X,y)
model.predict(X.loc[[0]]) # INPUT NEEDS TO BE A DATAFRAME


0        2
1        2
2        3
3        3
4        3
        ..
85484    1
85485    2
85486    2
85487    1
85488    2
Name: SLON3_avg, Length: 85489, dtype: int64

In [440]:
y

0        2
1        2
2        3
3        3
4        3
        ..
85484    1
85485    2
85486    2
85487    1
85488    2
Name: SLON3_avg, Length: 85489, dtype: int64